<a href="https://colab.research.google.com/github/martin-fabbri/tpu-workflow/blob/main/notebooks/tpu_checkpoints_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Save and load checkpoints

In [1]:
%%capture
!git clone https://github.com/martin-fabbri/tpu-workflow.git

In [2]:
%cd /content/tpu-workflow/

/content/tpu-workflow


In [3]:
%%capture
!pip install --upgrade --pre dvc

In [4]:
!pip list | grep "dvc\|tensorflow"

dvc                           2.0.0a2       
tensorflow                    2.4.1         
tensorflow-datasets           4.0.1         
tensorflow-estimator          2.4.0         
tensorflow-gcs-config         2.4.0         
tensorflow-hub                0.11.0        
tensorflow-metadata           0.28.0        
tensorflow-probability        0.12.1        


In [5]:
!dvc pull -r origin

Downloading:   0% 0/5 [00:00<?, ?file/s{'info': ''}]
!
plots/train_accuracy_plot.json          |0.00 [00:00,       ?it/s]
Downloading:  20% 1/5 [00:00<00:02,  1.83file/s{'info': ''}]
!
data/interim/train_split.json          |0.00 [00:00,       ?it/s]
                                                                 
!
data/interim/val_split.json          |0.00 [00:00,       ?it/s]
                                                               
!
data/interim/images_count.json          |0.00 [00:00,       ?it/s]
                                                                  
!
models/xception_model.h5          |0.00 [00:00,       ?it/s]
models/xception_model.h5          |259k [00:00,    2.64MB/s]
models/xception_model.h5          |1.51M [00:00,    8.85MB/s]
models/xception_model.h5          |7.84M [00:00,    35.1MB/s]
models/xception_model.h5          |14.9M [00:00,    50.5MB/s]
models/xception_model.h5          |25.6M [00:00,    72.8MB/s]
models/xception_model.h5          |32.6M [00:

In [ ]:
#!dvc repro

ERROR: failed to reproduce 'dvc.yaml': Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started


In [6]:
!pwd

/content/tpu-workflow


In [7]:
import os
import sys
from pathlib import Path

path = Path(os.getcwd())
src_path = str(path/"src")
sys.path.append(src_path)
sys.path
path

PosixPath('/content/tpu-workflow')

In [8]:
import json

import numpy as np
import tensorflow as tf
import yaml
from matplotlib import pyplot as plt
from models.xception_ft_model import create_xception_ft_model
from pipeline.train_utils import (
    MapDict,
    get_training_dataset,
    get_validation_dataset,
    load_dataset,
    lrfn,
    to_float32,
)
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

print("tensorflow", tf.__version__)

tensorflow 2.4.1


In [9]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.MirroredStrategy()
print("Number of accelerators", strategy.num_replicas_in_sync)

INFO:tensorflow:Initializing the TPU system: grpc://10.15.132.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.15.132.18:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Number of accelerators 8


## Load config

In [10]:
params_path = str(path/"params.yaml")

with open(params_path, "r") as params_file:
    params = yaml.load(params_file)
dataset_config = MapDict(params["dataset"])
split_config = MapDict(params["split"])
train_config = MapDict(params["train"])
train_config.batch_size *= strategy.num_replicas_in_sync
checkpoint_config = MapDict(params["checkpoint"]) 

lr_config = MapDict(params["train"]["lr"])
lr_config.max *= strategy.num_replicas_in_sync

print("dataset:\n\n", dataset_config)
print()
print("split:\n\n", split_config)
print()
print("train:\n\n", train_config)
print()
print("lr:\n\n", lr_config)
print()
print("checkpoint:\n\n", checkpoint_config)

dataset:

 {'image_size': 331, 'gcs_path': 'gs://kaggle-data-tpu/images', 'gcs_path_remote_alias': 'gcs_images', 'classes': ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips'], 'classes_path': 'data/domain/classes.json'}

split:

 {'train_split_path': 'data/interim/train_split.json', 'val_split_path': 'data/interim/val_split.json', 'images_count_path': 'data/interim/images_count.json', 'validation_split': 0.19}

train:

 {'nb_validation_samples': 800, 'epochs': 12, 'batch_size': 128, 'lr': {'start': 1e-05, 'max': 5e-05, 'min': 1e-05, 'rampup_epochs': 5, 'sustain_epochs': 0, 'exp_decay': 0.8}, 'saved_model_path': 'models/xception_model.h5', 'train_plot_path': 'plots/train_accuracy_plot.json'}

lr:

 {'start': 1e-05, 'max': 0.0004, 'min': 1e-05, 'rampup_epochs': 5, 'sustain_epochs': 0, 'exp_decay': 0.8}

checkpoint:

 {'path': 'checkpoints/cp.ckpt'}


In [ ]:
#@title ##Display images utils
def display_one_flower(image, title, subplot, red=False):
    plt.subplot(subplot)
    plt.axis('off')
    plt.imshow(image)
    plt.title(title, fontsize=16, color='red' if red else 'black')
    return subplot+1
    
def dataset_to_numpy_util(dataset, N):
    dataset = dataset.unbatch().batch(N)
    for images, labels in dataset:
        numpy_images = images.numpy()
        numpy_labels = labels.numpy()
        break;  
    return numpy_images, numpy_labels

def dataset_to_numpy_util(dataset, N):
    dataset = dataset.unbatch().batch(N)
    for images, labels in dataset:
        numpy_images = images.numpy()
        numpy_labels = labels.numpy()
        break;  
    return numpy_images, numpy_labels

def display_9_images_from_dataset(dataset):
    subplot=331
    plt.figure(figsize=(13,13))
    images, labels = dataset_to_numpy_util(dataset, 9)
    for i, image in enumerate(images):
        title = dataset_config.classes[np.argmax(labels[i], axis=-1)]
        subplot = display_one_flower(image, title, subplot)
        if i >= 8:
            break
              
    #plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()

def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        #plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

def title_from_label_and_target(label, correct_label):
    label = np.argmax(label, axis=-1)  # one-hot to class number
    correct_label = np.argmax(correct_label, axis=-1) # one-hot to class number
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(dataset_config.classes[label], str(correct), ', shoud be ' if not correct else '',
                                dataset_config.classes[correct_label] if not correct else ''), correct

def display_9_images_with_predictions(images, predictions, labels):
    subplot=331
    plt.figure(figsize=(13,13))
    for i, image in enumerate(images):
        title, correct = title_from_label_and_target(predictions[i], labels[i])
        subplot = display_one_flower(image, title, subplot, not correct)
        if i >= 8:
            break;
              
    #plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()

In [11]:
train_split_path = str(path / split_config.train_split_path)
val_split_path = str(path / split_config.val_split_path)

with open(train_split_path, "r") as train_split_file, open(val_split_path, "r") as val_split_file:
    train_split_files = json.load(train_split_file)
    val_split_files = json.load(val_split_file)

train_split_files, val_split_files

(['gs://kaggle-data-tpu/images/tfrecords-jpeg-331x331/flowers00-230.tfrec',
  'gs://kaggle-data-tpu/images/tfrecords-jpeg-331x331/flowers01-230.tfrec',
  'gs://kaggle-data-tpu/images/tfrecords-jpeg-331x331/flowers02-230.tfrec',
  'gs://kaggle-data-tpu/images/tfrecords-jpeg-331x331/flowers03-230.tfrec',
  'gs://kaggle-data-tpu/images/tfrecords-jpeg-331x331/flowers04-230.tfrec',
  'gs://kaggle-data-tpu/images/tfrecords-jpeg-331x331/flowers05-230.tfrec',
  'gs://kaggle-data-tpu/images/tfrecords-jpeg-331x331/flowers06-230.tfrec',
  'gs://kaggle-data-tpu/images/tfrecords-jpeg-331x331/flowers07-230.tfrec',
  'gs://kaggle-data-tpu/images/tfrecords-jpeg-331x331/flowers08-230.tfrec',
  'gs://kaggle-data-tpu/images/tfrecords-jpeg-331x331/flowers09-230.tfrec',
  'gs://kaggle-data-tpu/images/tfrecords-jpeg-331x331/flowers10-230.tfrec',
  'gs://kaggle-data-tpu/images/tfrecords-jpeg-331x331/flowers11-230.tfrec',
  'gs://kaggle-data-tpu/images/tfrecords-jpeg-331x331/flowers12-230.tfrec'],
 ['gs://kag

## Load training set images

In [12]:
training_dataset = get_training_dataset(
    train_split_files, dataset_config.image_size, train_config.batch_size
)

validation_dataset = get_validation_dataset(
    val_split_files, dataset_config.image_size, train_config.batch_size
)

training_dataset, validation_dataset

(<PrefetchDataset shapes: ((None, 331, 331, 3), (None, 5)), types: (tf.uint8, tf.float32)>,
 <PrefetchDataset shapes: ((None, 331, 331, 3), (None, 5)), types: (tf.uint8, tf.float32)>)

## Training

In [13]:
num_classes = len(dataset_config.classes)
with strategy.scope():
    model = create_xception_ft_model(dataset_config.image_size, num_classes)
model.summary()

83689472/83683744 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 331, 331, 3)       0         
_________________________________________________________________
xception (Functional)        (None, None, None, 2048)  20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 20,871,725
Trainable params: 20,817,197
Non-trainable params: 54,528
_________________________________________________________________


In [14]:
training_dataset = training_dataset.map(to_float32)
validation_dataset = validation_dataset.map(to_float32)

In [15]:
images_split_count_path = str(path/split_config.images_count_path)

with open(images_split_count_path, "r") as images_split_count_file:
    images_split_count = json.load(images_split_count_file)

num_training_images = images_split_count["num_training_images"]
num_validation_images = images_split_count["num_validation_images"]
num_training_images, num_validation_images

(2990, 680)

In [16]:
train_steps = num_training_images // train_config.batch_size
train_steps

23

In [17]:
lr_callback = LearningRateScheduler(
    lambda epoch: lrfn(lr_config, epoch), verbose=True
)

In [18]:
checkpoint_config.path

'checkpoints/cp.ckpt'

In [19]:
options = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")

In [20]:
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')

In [21]:
checkpoint_dir = os.path.dirname(checkpoint_config.path)
cp_callback = ModelCheckpoint(
    filepath=checkpoint_config.path,
    save_weights_only=True,
    verbose=1,
    options=options,
)
checkpoint_dir

'checkpoints'

In [22]:
history = model.fit(
    training_dataset,
    validation_data=validation_dataset,
    steps_per_epoch=train_steps,
    epochs=train_config.epochs,
    callbacks=[lr_callback, cp_callback]
)

Epoch 1/12

Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
23/23 [==============================] - 61s 776ms/step - loss: 1.5293 - accuracy: 0.3446 - val_loss: 1.3961 - val_accuracy: 0.5515

Epoch 00001: saving model to checkpoints/cp.ckpt
Epoch 2/12

Epoch 00002: LearningRateScheduler reducing learning rate to 8.8e-05.
23/23 [==============================] - 6s 278ms/step - loss: 0.9850 - accuracy: 0.7594 - val_loss: 0.3369 - val_accuracy: 0.9088

Epoch 00002: saving model to checkpoints/cp.ckpt
Epoch 3/12

Epoch 00003: LearningRateScheduler reducing learning rate to 0.000166.
23/23 [==============================] - 6s 280ms/step - loss: 0.2971 - accuracy: 0.9111 - val_loss: 0.2245 - val_accuracy: 0.9206

Epoch 00003: saving model to checkpoints/cp.ckpt
Epoch 4/12

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000244.
23/23 [==============================] - 7s 289ms/step - loss: 0.1459 - accuracy: 0.9546 - val_loss: 0.1445 - val_accuracy: 0.94

In [23]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [27]:
with strategy.scope():
    cp_model = create_xception_ft_model(dataset_config.image_size, num_classes)

loss, acc = cp_model.evaluate(validation_dataset, verbose=2)
print(f"Untrained model, accuracy {acc * 100:5.2f}")

6/6 - 14s - loss: 1.6508 - accuracy: 0.1309
Restored model, accuracy 13.09


In [30]:
with strategy.scope():
    cp_model.load_weights(checkpoint_config.path, options=options)

loss, acc = cp_model.evaluate(validation_dataset, verbose=2)
print(f"Restored model, accuracy {acc * 100:5.2f}")

6/6 - 2s - loss: 0.1106 - accuracy: 0.9691
Restored model, accuracy 96.91


## Customized checkpoints

In [36]:
checkpoint_path = "checkpoints/cp-{epoch:04d}.ckpt"
print("batch size:", train_config.batch_size)
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    options=options,
    save_freq=3
)
with strategy.scope():
    custom_cp_model = create_xception_ft_model(
        dataset_config.image_size, num_classes
    )
    
    custom_cp_model.save_weights(
        checkpoint_path.format(epoch=0), options=options
    )

custom_cp_model.fit(
    training_dataset,
    validation_data=validation_dataset,
    steps_per_epoch=train_steps,
    epochs=train_config.epochs,
    callbacks=[lr_callback, cp_callback]
)


batch size: 128
Epoch 1/12

Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
 3/23 [==>...........................] - ETA: 4s - loss: 1.6222 - accuracy: 0.1957
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
 6/23 [======>.......................] - ETA: 19s - loss: 1.6134 - accuracy: 0.2109
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
 9/23 [==========>...................] - ETA: 18s - loss: 1.6080 - accuracy: 0.2226
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
12/23 [==============>...............] - ETA: 15s - loss: 1.6015 - accuracy: 0.2341
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
15/23 [==================>...........] - ETA: 11s - loss: 1.5940 - accuracy: 0.2479
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
18/23 [======================>.......] - ETA: 7s - loss: 1.5862 - accuracy: 0.2616
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
21/23 [==========================>...] - ETA: 3s - loss: 1.5785 - accuracy: 0.

In [37]:
!ls {checkpoint_dir}

checkpoint			  cp-0007.ckpt.data-00000-of-00001
cp-0000.ckpt.data-00000-of-00001  cp-0007.ckpt.index
cp-0000.ckpt.index		  cp-0008.ckpt.data-00000-of-00001
cp-0001.ckpt.data-00000-of-00001  cp-0008.ckpt.index
cp-0001.ckpt.index		  cp-0009.ckpt.data-00000-of-00001
cp-0002.ckpt.data-00000-of-00001  cp-0009.ckpt.index
cp-0002.ckpt.index		  cp-0010.ckpt.data-00000-of-00001
cp-0003.ckpt.data-00000-of-00001  cp-0010.ckpt.index
cp-0003.ckpt.index		  cp-0011.ckpt.data-00000-of-00001
cp-0004.ckpt.data-00000-of-00001  cp-0011.ckpt.index
cp-0004.ckpt.index		  cp-0012.ckpt.data-00000-of-00001
cp-0005.ckpt.data-00000-of-00001  cp-0012.ckpt.index
cp-0005.ckpt.index		  cp.ckpt.data-00000-of-00001
cp-0006.ckpt.data-00000-of-00001  cp.ckpt.index
cp-0006.ckpt.index


In [39]:
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
latest_checkpoint

'checkpoints/cp-0012.ckpt'

## Restore latest checkpoint

In [41]:
with strategy.scope():
    latest_cp_model = create_xception_ft_model(
        dataset_config.image_size, num_classes
    )

loss, acc = latest_cp_model.evaluate(validation_dataset, verbose=2)
print(f"Untrained model, accuracy {acc * 100:5.2f}")

6/6 - 13s - loss: 1.6233 - accuracy: 0.1956
Untrained model, accuracy 19.56


In [42]:
with strategy.scope():
    latest_cp_model.load_weights(checkpoint_config.path, options=options)

loss, acc = latest_cp_model.evaluate(validation_dataset, verbose=2)
print(f"Restored model, accuracy {acc * 100:5.2f}")

6/6 - 2s - loss: 0.1106 - accuracy: 0.9691
Restored model, accuracy 96.91
